# Creating our model

Defining training set

In [ ]:
# labels / y
ratings = [
    4, 1, 8, 1, 7, 9, 10, 18, 4, 3, 13, 3, 7, 10, 3, 4, 7, 5, 6, 12,
    16, 10, 5, 10, 20, 16, 19, 19, 5, 0, 2, 3, 4, 7, 5, 2, 8, 7, 5, 5,
    5, 11, 0, 3, 1, 3, 6, 4, 1, 1, 5, 5, 3, 6, 11, 6, 6, 8, 1, 10,
    3, 5, 10, 0, 1, 0, 0, 3, 5, 3, 12, 3, 1, 1, 2, 6, 2, 0, 0, 6,
    6, 3, 3, 4, 7, 8, 6, 0, 0, 6, 6, 13, 0, 14, 1, 5, 0, 0, 0, 0,
    0, 0, 4, 4, 9, 6, 6, 0, 2, 6, 2, 7, 6, 0, 1, 0, 6, 5, 3, 3,
    18, 17, 10, 10, 10, 11, 19, 17, 19, 7, 4, 19, 13, 6, 5, 5, 2, 1,
    0, 0, 3, 3, 1, 7, 7, 7, 3, 8, 5, 4, 6, 5, 3, 5, 8, 5, 6, 11,
    0, 0, 4, 10, 13, 9, 12, 16, 12, 5, 3, 2, 1, 3, 3, 1, 4, 4, 17, 17,
    19, 14, 15, 13, 10, 9, 17, 19, 11, 17, 9
]

# images / X
training_images = []
for i in range(0,189):
  training_images.append(str(i)+".png")

Data preparation

In [ ]:
import numpy as np
import cv2

X = []
for file_name in training_images:
    img = cv2.imread(f"data/{file_name}")
    img = cv2.resize(img, (256, 256))
    X.append(img)

X = np.array(X) / 255
y = np.array(ratings)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

Data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2
)

augment_factor = 4
augmented_images = []
augmented_labels = []
for x, y in zip(X_train, y_train):
    augmented_images.append(x)
    augmented_labels.append(y)
    for _ in range(augment_factor - 1):
        transformed_image = datagen.random_transform(x)
        augmented_images.append(transformed_image)
        augmented_labels.append(y)

augmented_X_train = np.array(augmented_images)
augmented_y_train = np.array(augmented_labels)

Creating our model

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential

def create_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(256,256, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(256, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.2),
        Dense(1)
        ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = KerasClassifier(build_fn=create_model, epochs=12, batch_size=32)


Training

In [ ]:
model.fit(augmented_X_train, augmented_y_train, validation_data=(X_test, y_test))

Benchmarking


In [ ]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 37ms/step - loss: 1.2789


1.278922200202942

Exporting

In [ ]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()

open("model.tflite" , "wb").write(tfmodel)

# Importing & using our model


In [ ]:
import joblib

loaded_model = joblib.load('model.pkl')

In [ ]:
import numpy as np
import cv2

img = cv2.imread("yourImage")
img = cv2.resize(img, (256, 256))
img = np.array(img) / 255
img = np.expand_dims(img, axis=0)
results = round(float(model.predict(img)[0][0]), 1)
results